# Model agnostic grid operations using `xgcm`

In [1]:
import xarray as xr
import xgcm
import matplotlib.pyplot as plt
import dask
import cmocean
import gsw
import cosima_cookbook as cc
import numpy as np

%matplotlib inline



In [2]:
# fire up a Dask cluster using the cores hosting the notebook
# If you need more oomph, NCI has instructions for creating a PBS cluster than can use multiple nodes
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()

2023-01-24 10:59:00,565 - distributed.diskutils - INFO - Found stale lock file and directory '/jobfs/70723530.gadi-pbs/dask-worker-space/worker-rcb4124n', purging
2023-01-24 10:59:00,565 - distributed.diskutils - INFO - Found stale lock file and directory '/jobfs/70723530.gadi-pbs/dask-worker-space/worker-fug_774z', purging
2023-01-24 10:59:00,566 - distributed.diskutils - INFO - Found stale lock file and directory '/jobfs/70723530.gadi-pbs/dask-worker-space/worker-kgvq7hew', purging
2023-01-24 10:59:00,566 - distributed.diskutils - INFO - Found stale lock file and directory '/jobfs/70723530.gadi-pbs/dask-worker-space/worker-qgwik044', purging
2023-01-24 10:59:00,566 - distributed.diskutils - INFO - Found stale lock file and directory '/jobfs/70723530.gadi-pbs/dask-worker-space/worker-j9y7t5sn', purging
2023-01-24 10:59:00,566 - distributed.diskutils - INFO - Found stale lock file and directory '/jobfs/70723530.gadi-pbs/dask-worker-space/worker-q0060pi0', purging
2023-01-24 10:59:00,56

In [3]:
# either way, run this
client = Client(cluster)


In [4]:
cluster

In [5]:
print(client)

<Client: 'tcp://127.0.0.1:36139' processes=7 threads=7, memory=32.00 GiB>


# Start a COSIMA cookbook session

In [6]:
session = cc.database.create_session()#'/g/data/ik11/databases/daily/cosima_master_2022-08-30.db')

# reference density value:
rho_0 = 1035.0
g = 9.81



In [7]:
sim = '0.1RYF'

if sim == '0.1RYF':
    # RYF run with daily outputs
    #   for years 2170-2179, there is global daily temp, salt, pot_rho_1, uhrho_et, vhrho_nt, u, v, wt, dzt.
    expt = '01deg_jra55v13_ryf9091'
    start_time = '2170-01-01'
    end_time = '2170-1-31'

else:
    raise ValueError("'sim' not set correctly - notebook only works with 0.1RYF")

In [8]:
time_slice = slice(start_time, end_time)


# Load some coordinates

In [9]:
%%time
#load coordinates

ocean_grid = xr.open_dataset('/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf9091/output000/ocean/ocean_grid.nc')

#load dzt
dzt = cc.querying.getvar(expt, 'dzt', session, start_time=start_time, end_time=end_time, ncfile='%daily%')


CPU times: user 6.61 s, sys: 1.91 s, total: 8.51 s
Wall time: 17.4 s


In [10]:

st_ocean = cc.querying.getvar(expt, 'st_ocean', session, start_time=start_time, end_time=end_time,  ncfile='%daily%')
ds_st_edges_ocean = cc.querying.getvar(expt, 'st_edges_ocean', session, start_time=start_time, end_time=end_time,  ncfile='%daily%')


In [11]:
ocean_grid

<xarray.Dataset>
Dimensions:     (xt_ocean: 3600, yt_ocean: 2700, time: 1, xu_ocean: 3600,
                 yu_ocean: 2700)
Coordinates:
  * xt_ocean    (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean    (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
  * time        (time) object 1900-04-01 00:00:00
  * xu_ocean    (xu_ocean) float64 -279.9 -279.8 -279.7 ... 79.8 79.9 80.0
  * yu_ocean    (yu_ocean) float64 -81.09 -81.05 -81.0 ... 89.92 89.96 90.0
    geolon_t    (yt_ocean, xt_ocean) float32 ...
    geolat_t    (yt_ocean, xt_ocean) float32 ...
    geolon_c    (yu_ocean, xu_ocean) float32 ...
    geolat_c    (yu_ocean, xu_ocean) float32 ...
Data variables:
    ht          (yt_ocean, xt_ocean) float32 ...
    hu          (yu_ocean, xu_ocean) float32 ...
    dxt         (yt_ocean, xt_ocean) float32 ...
    dyt         (yt_ocean, xt_ocean) float32 ...
    dxu         (yu_ocean, xu_ocean) float32 ...
    dyu         (yu_ocean, xu_ocean) float32 ...
    area_t      (yt_ocean, xt_ocean) float32 ...
    area_u      (yu_ocean, xu_ocean) float32 ...
    kmt         (yt_ocean, xt_ocean) float32 ...
    kmu         (yu_ocean, xu_ocean) float32 ...
    drag_coeff  (time, yu_ocean, xu_ocean) float32 ...
Attributes:
    filename:   ocean_grid.nc
    title:      ACCESS-OM2-01
    grid_type:  mosaic
    grid_tile:  1

# Load some variables

In [12]:
%%time

#load pot_rho_1
pot_rho_1 = cc.querying.getvar(expt,'pot_rho_1',session,start_time=start_time, end_time=end_time,ncfile='%daily%')

u = cc.querying.getvar(expt,'u',session,start_time=start_time, end_time=end_time,ncfile='%daily%')
v = cc.querying.getvar(expt,'v',session,start_time=start_time, end_time=end_time,ncfile='%daily%')

time = pot_rho_1.time


CPU times: user 11.1 s, sys: 608 ms, total: 11.7 s
Wall time: 16.1 s


In [27]:
u.ncfiles[0]

'/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf9091/output1076/ocean/ocean_daily_3d_u.nc'

In [14]:
ocean = xr.open_dataset('/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf9091/output1077/ocean/ocean.nc')

In [15]:
ocean

<xarray.Dataset>
Dimensions:                (xt_ocean: 3600, yt_ocean: 2700, st_ocean: 75,
                            st_edges_ocean: 76, time: 3, nv: 2, xu_ocean: 3600,
                            yu_ocean: 2700, sw_ocean: 75, sw_edges_ocean: 76,
                            grid_xu_ocean: 3600, grid_yt_ocean: 2700,
                            potrho: 80, potrho_edges: 81, grid_xt_ocean: 3600,
                            grid_yu_ocean: 2700, neutral: 80,
                            neutralrho_edges: 81)
Coordinates: (12/18)
  * xt_ocean               (xt_ocean) float64 -279.9 -279.8 ... 79.85 79.95
  * yt_ocean               (yt_ocean) float64 -81.11 -81.07 ... 89.94 89.98
  * st_ocean               (st_ocean) float64 0.5413 1.681 ... 5.709e+03
  * st_edges_ocean         (st_edges_ocean) float64 0.0 1.083 ... 5.809e+03
  * time                   (time) object 2170-04-16 00:00:00 ... 2170-06-16 0...
  * nv                     (nv) float64 1.0 2.0
    ...                     ...
  * potrho                 (potrho) float64 1.028e+03 1.028e+03 ... 1.038e+03
  * potrho_edges           (potrho_edges) float64 1.028e+03 ... 1.038e+03
  * grid_xt_ocean          (grid_xt_ocean) float64 -279.9 -279.8 ... 79.85 79.95
  * grid_yu_ocean          (grid_yu_ocean) float64 -81.09 -81.05 ... 89.96 90.0
  * neutral                (neutral) float64 1.028e+03 1.028e+03 ... 1.038e+03
  * neutralrho_edges       (neutralrho_edges) float64 1.028e+03 ... 1.038e+03
Data variables: (12/28)
    temp                   (time, st_ocean, yt_ocean, xt_ocean) float32 ...
    pot_temp               (time, st_ocean, yt_ocean, xt_ocean) float32 ...
    salt                   (time, st_ocean, yt_ocean, xt_ocean) float32 ...
    age_global             (time, st_ocean, yt_ocean, xt_ocean) float32 ...
    u                      (time, st_ocean, yu_ocean, xu_ocean) float32 ...
    v                      (time, st_ocean, yu_ocean, xu_ocean) float32 ...
    ...                     ...
    bih_fric_v             (time, st_ocean, yu_ocean, xu_ocean) float32 ...
    u_dot_grad_vert_pv     (time, st_ocean, yt_ocean, xt_ocean) float32 ...
    average_T1             (time) datetime64[ns] ...
    average_T2             (time) datetime64[ns] ...
    average_DT             (time) timedelta64[ns] ...
    time_bounds            (time, nv) timedelta64[ns] ...
Attributes:
    filename:   ocean.nc
    title:      ACCESS-OM2-01
    grid_type:  mosaic
    grid_tile:  1

# Create an `xgcm` grid object

In [31]:
ds = xr.merge([u, v, pot_rho_1])

In [32]:
ds

<xarray.Dataset>
Dimensions:    (xu_ocean: 3600, yu_ocean: 2700, st_ocean: 75, time: 273,
                xt_ocean: 3600, yt_ocean: 2700)
Coordinates:
  * xu_ocean   (xu_ocean) float64 -279.9 -279.8 -279.7 -279.6 ... 79.8 79.9 80.0
  * yu_ocean   (yu_ocean) float64 -81.09 -81.05 -81.0 ... 89.92 89.96 90.0
  * st_ocean   (st_ocean) float64 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
  * time       (time) object 2170-01-01 12:00:00 ... 2170-09-30 12:00:00
  * xt_ocean   (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean   (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
Data variables:
    u          (time, st_ocean, yu_ocean, xu_ocean) float32 dask.array<chunksize=(1, 7, 300, 400), meta=np.ndarray>
    v          (time, st_ocean, yu_ocean, xu_ocean) float32 dask.array<chunksize=(1, 7, 300, 400), meta=np.ndarray>
    pot_rho_1  (time, st_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(1, 7, 300, 400), meta=np.ndarray>
Attributes: (12/13)
    long_name:      i-current
    units:          m/sec
    valid_range:    [-10.  10.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_c geolat_c
    ...             ...
    ncfiles:        ['/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf90...
    contact:        Andy Hogg
    email:          andy.hogg@anu.edu.au
    created:        2020-06-11
    description:    0.1 degree ACCESS-OM2 global model configuration under th...
    notes:          Additional daily outputs saved from 1 Jan 1950 to 31 Dec ...

In [44]:
ds = ds.assign_coords({'st_ocean':ocean['st_ocean'], 'st_edges_ocean':ocean['st_edges_ocean'],
                      'dxu':ocean_grid['dxu'], 'dxt':ocean_grid['dxt'],
                      'dyu':ocean_grid['dyu'], 'dyt':ocean_grid['dyt'],
                      'area_t':ocean_grid['area_t'], 'area_u':ocean_grid['area_u'],})

In [45]:
ds

<xarray.Dataset>
Dimensions:         (xu_ocean: 3600, yu_ocean: 2700, st_ocean: 75, time: 273,
                     xt_ocean: 3600, yt_ocean: 2700, st_edges_ocean: 76)
Coordinates: (12/18)
  * xu_ocean        (xu_ocean) float64 -279.9 -279.8 -279.7 ... 79.8 79.9 80.0
  * yu_ocean        (yu_ocean) float64 -81.09 -81.05 -81.0 ... 89.92 89.96 90.0
  * st_ocean        (st_ocean) float64 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
  * time            (time) object 2170-01-01 12:00:00 ... 2170-09-30 12:00:00
  * xt_ocean        (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.85 79.95
  * yt_ocean        (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.94 89.98
    ...              ...
    dxt             (yt_ocean, xt_ocean) float32 ...
    dyu             (yu_ocean, xu_ocean) float32 ...
    dyt             (yt_ocean, xt_ocean) float32 ...
    area_t          (yt_ocean, xt_ocean) float32 ...
    are_u           (yu_ocean, xu_ocean) float32 ...
    area_u          (yu_ocean, xu_ocean) float32 ...
Data variables:
    u               (time, st_ocean, yu_ocean, xu_ocean) float32 dask.array<chunksize=(1, 7, 300, 400), meta=np.ndarray>
    v               (time, st_ocean, yu_ocean, xu_ocean) float32 dask.array<chunksize=(1, 7, 300, 400), meta=np.ndarray>
    pot_rho_1       (time, st_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(1, 7, 300, 400), meta=np.ndarray>
Attributes: (12/13)
    long_name:      i-current
    units:          m/sec
    valid_range:    [-10.  10.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_c geolat_c
    ...             ...
    ncfiles:        ['/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf90...
    contact:        Andy Hogg
    email:          andy.hogg@anu.edu.au
    created:        2020-06-11
    description:    0.1 degree ACCESS-OM2 global model configuration under th...
    notes:          Additional daily outputs saved from 1 Jan 1950 to 31 Dec ...

In [46]:

metrics = {
    ('X',): ['dxu', 'dxt'], # X distances
    ('Y',): ['dyu', 'dyt'], # Y distances
    # ('Z',): ['dzt'], # Z distances (varies in time because of MOM's vetical coordinate)
    ('X', 'Y'): ['area_t', 'area_u'] # Areas
}
grid = xgcm.Grid(ds, coords={'X':{'center':'xt_ocean', 'right':'xu_ocean'},
                             'Y':{'center':'yt_ocean', 'right':'yu_ocean'},
                             'Z':{'center':'st_ocean', 'outer':'st_edges_ocean'}},
                 periodic = ['X'], metrics=metrics)



### Note
`st_ocean` is the tracer cell depth and `sw_ocean` is the vertical velocity cell depth. `st_edges_ocean` contains all the values in `sw_ocean` with a 0 appended to the beginning.

In [25]:
grid

<xgcm.Grid>
X Axis (periodic, boundary=None):
  * center   xt_ocean --> right
  * right    xu_ocean --> center
Y Axis (not periodic, boundary=None):
  * center   yt_ocean --> right
  * right    yu_ocean --> center
Z Axis (not periodic, boundary=None):
  * center   st_ocean --> outer
  * outer    st_edges_ocean --> center

# Do some operations

MOM5 computes relative vorticity as:

```
vorticity_z =
onehalf*(
(v(i,j,k) - v(i-1,j,k) )*Grd%dxtnr(i,j) &
+(v(i,j-1,k) - v(i-1,j-1,k))*Grd%dxtnr(i,j-1) &
-(u(i,j,k) - u(i,j-1,k) )*Grd%dyter(i,j) &
-(u(i-1,j,k) - u(i-1,j-1,k))*Grd%dyter(i-1,j)
)
```

See https://github.com/COSIMA/cosima-recipes/issues/13#issuecomment-311843830

In [54]:
rel_vort = grid.interp(grid.derivative(v, 'X'), 'Y') - grid.interp(grid.derivative(u, 'Y'), 'X')

In [55]:
rel_vort

<xarray.DataArray (time: 273, st_ocean: 75, yt_ocean: 2700, xt_ocean: 3600)>
dask.array<sub, shape=(273, 75, 2700, 3600), dtype=float32, chunksize=(1, 7, 300, 400), chunktype=numpy.ndarray>
Coordinates:
  * st_ocean  (st_ocean) float64 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
  * time      (time) object 2170-01-01 12:00:00 ... 2170-09-30 12:00:00
  * xt_ocean  (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98

In [ ]:
rel_vort.sel(time='2170-01-10').sel(st_ocean=10, method='nearest').plot()

2023-01-24 11:46:28,511 - tornado.application - ERROR - Exception in callback <bound method BokehTornado._keep_alive of <bokeh.server.tornado.BokehTornado object at 0x148db02fe040>>
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/site-packages/tornado/ioloop.py", line 905, in _run
    return self.callback()
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/site-packages/bokeh/server/tornado.py", line 760, in _keep_alive
    c.send_ping()
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/site-packages/bokeh/server/connection.py", line 93, in send_ping
    self._socket.ping(str(self._ping_count).encode("utf-8"))
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-22.07/lib/python3.9/site-packages/tornado/websocket.py", line 445, in ping
    raise WebSocketClosedError()
tornado.websocket.WebSocketClosedError
2023-01-24 11:46:32,389 - tornado.application - ERROR - 